**Observer** is one of the behavioral design patterns that talks about how to have multiple clients connected to one object and when the object is updated we want to tell other clinets connected to the obj to know about these changes <br>
we have publisher/ subject/observerable who is responsible to create that event and send it to others<br>
we have subscribers who each of them have an update method which publisher calls it.<br>


In [ ]:
# structure
from abc import ABC, abstractmethod
class Publisher:
  def __init__(self):
    self.observers=set()
  def add_observer(self,observer):
    self.observers.add(observer)
  def remove_observer(self,observer):
    self.observers.remove(observer)
  def notify_observers(self,message):
    for observer in self.observers:
      observer.update(message)

class ObserverInterface(ABC):
  @abstractmethod
  def update(self,message):
    pass

class Observer1(ObserverInterface):
  def update(self,message):
    print(f"Observer1 received message: {message}")

class Observer2(ObserverInterface):
  def update(self,message):
    print(f"Observer2 received message: {message}")

publisher=Publisher()
observer1=Observer1()
observer2=Observer2()
publisher.add_observer(observer1)
publisher.add_observer(observer2)
publisher.notify_observers("Hello, world!")

Observer2 received message: Hello, world!
Observer1 received message: Hello, world!


**Example 1**

In [ ]:
# awful code
class DataSource:
    def __init__(self):
        self.data = None

    def set_data(self, data):
        self.data = data
        # Directly updating the UI
        display.update_display(data)
        logger.log_data_change(data)

class Display:
    def update_display(self, data):
        print(f"Displaying: {data}")

class Logger:
    def log_data_change(self, data):
        print(f"Logging: {data}")

# Example Usage
if __name__ == "__main__":
    data_source = DataSource()
    display = Display()
    logger = Logger()

    data_source.set_data("New Data")


Displaying: New Data
Logging: New Data


In [ ]:
# refactored
from abc import ABC, abstractmethod

class PublisherDataSource:
    def __init__(self):
        self._observers = set()
        self._data = None

    def add_observer(self, observer):
        self._observers.add(observer)

    def remove_observer(self, observer):
        self._observers.remove(observer)

    def notify_observers(self):
        for observer in self._observers:
            observer.update(self._data)

    def set_data(self, data):
        self._data = data
        self.notify_observers()

class ObserverInterface(ABC):
    @abstractmethod
    def update(self, data):
        pass

class Display(ObserverInterface):
    def update(self, data):
        self.update_display(data)

    def update_display(self, data):
        print(f"Displaying: {data}")

class Logger(ObserverInterface):
    def update(self, data):
        self.log_data_change(data)

    def log_data_change(self, data):
        print(f"Logging: {data}")

# Example Usage
if __name__ == "__main__":
    data_source = PublisherDataSource()
    display = Display()
    logger = Logger()

    data_source.add_observer(display)
    data_source.add_observer(logger)

    data_source.set_data("New Data")


Displaying: New Data
Logging: New Data


** Example 2**


In [ ]:
# awful code
class OrderProcessor:
    def process_order(self, order_data):
        print(f"Processing order for {order_data['customer_name']}")

        # Directly sending confirmation email
        self.send_confirmation_email(order_data)

        # Directly updating inventory
        self.update_inventory(order_data)

        # Directly logging the order
        self.log_order(order_data)

    def send_confirmation_email(self, order_data):
        print(f"Sending email to {order_data['customer_email']} for order {order_data['order_id']}")

    def update_inventory(self, order_data):
        print(f"Updating inventory for items in order {order_data['order_id']}")

    def log_order(self, order_data):
        print(f"Logging order {order_data['order_id']} for customer {order_data['customer_name']}")

# Example Usage
if __name__ == "__main__":
    order_processor = OrderProcessor()

    order_data = {
        "order_id": "12345",
        "customer_name": "John Doe",
        "customer_email": "johndoe@example.com",
        "items": ["item1", "item2", "item3"]
    }

    order_processor.process_order(order_data)


Processing order for John Doe
Sending email to johndoe@example.com for order 12345
Updating inventory for items in order 12345
Logging order 12345 for customer John Doe


In [2]:
# refactored code
from abc import ABC, abstractmethod

class OrderProcessorPublisher:
    def __init__(self):
        self._observers = set()

    def add_observer(self, observer):
        self._observers.add(observer)

    def remove_observer(self, observer):
        self._observers.remove(observer)

    def notify_observers(self, order_data):
        for observer in self._observers:
            observer.update(order_data)

    def process_order(self, order_data):
        print(f"Processing order {order_data['order_id']} for {order_data['customer_name']}")
        self.notify_observers(order_data)

class ObserverInterface(ABC):
    @abstractmethod
    def update(self, order_data):
        pass

class EmailNotifier(ObserverInterface):
    def update(self, order_data):
        print(f"Sending email to {order_data['customer_email']} for order {order_data['order_id']}")

class OrderLogger(ObserverInterface):
    def update(self, order_data):
        print(f"Logging order {order_data['order_id']} for customer {order_data['customer_name']}")

class InventoryManager(ObserverInterface):
    def update(self, order_data):
        print(f"Updating inventory for items in order {order_data['order_id']}")

# Example Usage
if __name__ == "__main__":
    order_processor = OrderProcessorPublisher()

    order_data = {
        "order_id": "12345",
        "customer_name": "John Doe",
        "customer_email": "johndoe@example.com",
        "items": ["item1", "item2", "item3"]
    }

    # Create observers
    email_notifier = EmailNotifier()
    order_logger = OrderLogger()
    inventory_manager = InventoryManager()

    # Attach observers to the order processor
    order_processor.add_observer(email_notifier)
    order_processor.add_observer(order_logger)
    order_processor.add_observer(inventory_manager)

    # Process the order
    order_processor.process_order(order_data)


Processing order 12345 for John Doe
Logging order 12345 for customer John Doe
Updating inventory for items in order 12345
Sending email to johndoe@example.com for order 12345
